In [19]:
import numpy as np # linear algebra
import scipy as sp 
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from sklearn.linear_model import LinearRegression
import time
import logging
import datetime
import warnings
from scipy import stats
from tqdm import tqdm_notebook
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings('ignore')
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
from scipy import stats

import os
print(os.listdir("./"))

['.ipynb_checkpoints', 'earthquake.ipynb', 'LANL-Earthquake-Prediction.zip', 'S1.png', 's2.png', 'sample_submission.csv', 'test.zip', 'train.csv', '確定地震可能即將來臨的時間.docx']


先簡單看前10000000筆的資料

In [2]:
train = pd.read_csv("train.csv", nrows=10000000,
                    dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})
train.head(5)

,acoustic_data,time_to_failure
0,12,1.4691
1,6,1.4691
2,8,1.4691
3,5,1.4691
4,8,1.4691


In [3]:
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', '{:10,.10f}'.format)
pd.set_option('display.max_colwidth', -1)

In [4]:
train.head(5)

,acoustic_data,time_to_failure
0,12,1.4690999832
1,6,1.4690999821
2,8,1.4690999810
3,5,1.4690999799
4,8,1.4690999788


可以觀察到time_to_failure隨著index增加而減少，表示越接近地震的發生。

從kaggle中有kernel顯示，在training set中總共發生16次地震，而且在time_to_failure = 0 時發生。

In [5]:
train.describe()

,acoustic_data,time_to_failure
count,"10,000,000.0000000000","10,000,000.0000000000"
mean,4.5020723000,5.1835978573
std,17.8070724662,5.0912858694
min,"-4,621.0000000000",0.0007954798
25%,2.0000000000,0.6498970646
50%,4.0000000000,1.2988986484
75%,7.0000000000,10.8916984033
max,"3,252.0000000000",11.5407999870


In [6]:
%%time
train_df = pd.read_csv(os.path.join('train.csv'), dtype={'acoustic_data': np.int16, 'time_to_failure': np.float32})

Wall time: 24min 46s


以下函數是生成1、2、3......的這種基本數字序列(長度和輸入的數據長度相同)，並且和輸入的數據做線性回歸分析，並且返回變量係數。

In [8]:
def add_trend_feature(arr, abs_values=False):
    idx = np.array(range(len(arr)))
    if abs_values:
        arr = np.abs(arr)
    lr = LinearRegression()
    lr.fit(idx.reshape(-1, 1), arr)
    return lr.coef_[0]

以下函數是用來計算sta和Ita的筆直，這兩個數據在地球物理中用來預測地震。

In [9]:
def classic_sta_lta(x, length_sta, length_lta):   
    sta = np.cumsum(x ** 2)
    # Convert to float
    sta = np.require(sta, dtype=np.float)
    # Copy for LTA
    lta = sta.copy()
    # Compute the STA and the LTA
    sta[length_sta:] = sta[length_sta:] - sta[:-length_sta]
    sta /= length_sta
    lta[length_lta:] = lta[length_lta:] - lta[:-length_lta]
    lta /= length_lta
    # Pad zeros
    sta[:length_lta - 1] = 0
    # Avoid division by zero by setting zero values to tiny float
    dtiny = np.finfo(0.0).tiny
    idx = lta < dtiny
    lta[idx] = dtiny
    return sta / lta

In [12]:
def calc_change_rate(x):
    change = (np.diff(x) / x[:-1]).values
    change = change[np.nonzero(change)[0]]
    change = change[~np.isnan(change)]
    change = change[change != -np.inf]
    change = change[change != np.inf]
    return np.mean(change)

In [10]:
rows = 150000
segments = int(np.floor(train_df.shape[0] / rows))

In [11]:
train_X = pd.DataFrame(index=range(segments), dtype=np.float64)
train_y = pd.DataFrame(index=range(segments), dtype=np.float64, columns=['time_to_failure'])

In [17]:
def create_features(seg_id, seg, X):
    xc = pd.Series(seg['acoustic_data'].values)   
    zc = np.fft.fft(xc) #快速傅立葉變换
    realFFT = np.real(zc)# 獲取實數部分
    imagFFT = np.imag(zc)#獲取虛數部分
    X.loc[seg_id, 'mean'] = xc.mean()
    X.loc[seg_id, 'std'] = xc.std()#標準差
    X.loc[seg_id, 'max'] = xc.max()
    X.loc[seg_id, 'min'] = xc.min()
    X.loc[seg_id, 'sum'] = xc.sum()
    X.loc[seg_id, 'mad'] = xc.mad()#根據平均值計算平均絕對離差
    X.loc[seg_id, 'kurt'] = xc.kurtosis()#峰度
    X.loc[seg_id, 'skew'] = xc.skew()#偏度，是統計數據分布偏斜方向和程度的度量，是統計數據分布非對稱程度的數字特徵。
   
    X.loc[seg_id, 'max_to_min'] = xc.max() / np.abs(xc.min())
    X.loc[seg_id, 'max_to_min_diff'] = xc.max() - np.abs(xc.min())
    X.loc[seg_id, 'count_big'] = len(xc[np.abs(xc) > 500])

    X.loc[seg_id, 'med'] = xc.median()#算術中位數(50%分位數) 
    X.loc[seg_id, 'abs_mean'] = np.abs(xc).mean()#xc的絕對值，之後再取平均值
    X.loc[seg_id, 'abs_max'] = np.abs(xc).max()
    X.loc[seg_id, 'abs_min'] = np.abs(xc).min()    
    X.loc[seg_id, 'mean_change_abs'] = np.mean(np.diff(xc))
    X.loc[seg_id, 'mean_change_rate'] = calc_change_rate(xc)
    X.loc[seg_id, 'ave10'] = stats.trim_mean(xc, 0.1)

    #分位數之統計
    X.loc[seg_id, 'q95'] = np.quantile(xc, 0.95)#95%分位數
    X.loc[seg_id, 'q99'] = np.quantile(xc, 0.99)#99%分位數
    X.loc[seg_id, 'q05'] = np.quantile(xc, 0.05)#5%分位數
    X.loc[seg_id, 'q01'] = np.quantile(xc, 0.01)#1%分位數
    X.loc[seg_id, 'q999'] = np.quantile(xc,0.999)#99.9%分位數
    X.loc[seg_id, 'q001'] = np.quantile(xc,0.001)#0.1%分位數
    X.loc[seg_id, 'iqr'] = np.subtract(*np.percentile(xc, [75, 25]))
    
    X.loc[seg_id, 'abs_q95'] = np.quantile(np.abs(xc), 0.95)
    X.loc[seg_id, 'abs_q99'] = np.quantile(np.abs(xc), 0.99)
    X.loc[seg_id, 'abs_q05'] = np.quantile(np.abs(xc), 0.05)
    X.loc[seg_id, 'abs_q01'] = np.quantile(np.abs(xc), 0.01)
    
    #傅立葉變換之統計
    X.loc[seg_id, 'Rmean'] = realFFT.mean()
    X.loc[seg_id, 'Rstd'] = realFFT.std()
    X.loc[seg_id, 'Rmax'] = realFFT.max()
    X.loc[seg_id, 'Rmin'] = realFFT.min()
    X.loc[seg_id, 'Imean'] = imagFFT.mean()
    X.loc[seg_id, 'Istd'] = imagFFT.std()
    X.loc[seg_id, 'Imax'] = imagFFT.max()
    X.loc[seg_id, 'Imin'] = imagFFT.min()
    #分段
    X.loc[seg_id, 'std_first_50000'] = xc[:50000].std()
    X.loc[seg_id, 'std_last_50000'] = xc[-50000:].std()
    X.loc[seg_id, 'std_first_25000'] = xc[:25000].std()
    X.loc[seg_id, 'std_last_25000'] = xc[-25000:].std()
    X.loc[seg_id, 'std_first_10000'] = xc[:10000].std()
    X.loc[seg_id, 'std_last_10000'] = xc[-10000:].std()
    
    X.loc[seg_id, 'avg_first_50000'] = xc[:50000].mean()
    X.loc[seg_id, 'avg_last_50000'] = xc[-50000:].mean()
    X.loc[seg_id, 'avg_first_25000'] = xc[:25000].mean()
    X.loc[seg_id, 'avg_last_25000'] = xc[-25000:].mean()
    X.loc[seg_id, 'avg_first_10000'] = xc[:10000].mean()
    X.loc[seg_id, 'avg_last_10000'] = xc[-10000:].mean()
    
    X.loc[seg_id, 'min_first_50000'] = xc[:50000].min()
    X.loc[seg_id, 'min_last_50000'] = xc[-50000:].min()
    X.loc[seg_id, 'min_first_25000'] = xc[:25000].min()
    X.loc[seg_id, 'min_last_25000'] = xc[-25000:].min()
    X.loc[seg_id, 'min_first_10000'] = xc[:10000].min()
    X.loc[seg_id, 'min_last_10000'] = xc[-10000:].min()
    
    X.loc[seg_id, 'max_first_50000'] = xc[:50000].max()
    X.loc[seg_id, 'max_last_50000'] = xc[-50000:].max()
    X.loc[seg_id, 'max_first_25000'] = xc[:25000].max()
    X.loc[seg_id, 'max_last_25000'] = xc[-25000:].max()
    X.loc[seg_id, 'max_first_10000'] = xc[:10000].max()
    X.loc[seg_id, 'max_last_10000'] = xc[-10000:].max()
    
    X.loc[seg_id, 'mean_change_rate_first_50000'] = calc_change_rate(xc[:50000])
    X.loc[seg_id, 'mean_change_rate_last_50000'] = calc_change_rate(xc[-50000:])
    X.loc[seg_id, 'mean_change_rate_first_25000'] = calc_change_rate(xc[:25000])
    X.loc[seg_id, 'mean_change_rate_last_25000'] = calc_change_rate(xc[-25000:])
    X.loc[seg_id, 'mean_change_rate_first_10000'] = calc_change_rate(xc[:10000])
    X.loc[seg_id, 'mean_change_rate_last_10000'] = calc_change_rate(xc[-10000:])
    
    X.loc[seg_id, 'Hilbert_mean'] = np.abs(hilbert(xc)).mean()#希爾伯特轉換，是一個對函數 u(t) 產生定義域相同的函數 H(u)(t) 的線性算子。 
    
    X.loc[seg_id, 'Hann_window_mean'] = (convolve(xc, hann(150), mode='same') / sum(hann(150))).mean()
    
    X.loc[seg_id, 'classic_sta_lta1_mean'] = classic_sta_lta(xc, 500, 10000).mean()
    X.loc[seg_id, 'classic_sta_lta2_mean'] = classic_sta_lta(xc, 5000, 100000).mean()
    X.loc[seg_id, 'classic_sta_lta3_mean'] = classic_sta_lta(xc, 3333, 6666).mean()
    X.loc[seg_id, 'classic_sta_lta4_mean'] = classic_sta_lta(xc, 10000, 25000).mean()
    X.loc[seg_id, 'classic_sta_lta5_mean'] = classic_sta_lta(xc, 50, 1000).mean()
    X.loc[seg_id, 'classic_sta_lta6_mean'] = classic_sta_lta(xc, 100, 5000).mean()
    X.loc[seg_id, 'classic_sta_lta7_mean'] = classic_sta_lta(xc, 333, 666).mean()
    X.loc[seg_id, 'classic_sta_lta8_mean'] = classic_sta_lta(xc, 4000, 10000).mean()
    
    X.loc[seg_id, 'Moving_average_700_mean'] = xc.rolling(window=700).mean().mean(skipna=True)
    
    ewma = pd.Series.ewm
    X.loc[seg_id, 'exp_Moving_average_300_mean'] = ewma(xc, span=300).mean().mean(skipna=True)
    X.loc[seg_id, 'exp_Moving_average_3000_mean'] = ewma(xc, span=3000).mean().mean(skipna=True)
    X.loc[seg_id, 'exp_Moving_average_30000_mean'] = ewma(xc, span=30000).mean().mean(skipna=True)
    
    no_of_std = 3
    X.loc[seg_id,'MA_700MA_std_mean'] = xc.rolling(window=700).std().mean()
    X.loc[seg_id,'MA_700MA_BB_high_mean'] = (X.loc[seg_id, 'Moving_average_700_mean'] + no_of_std * X.loc[seg_id, 'MA_700MA_std_mean']).mean()
    X.loc[seg_id,'MA_700MA_BB_low_mean'] = (X.loc[seg_id, 'Moving_average_700_mean'] - no_of_std * X.loc[seg_id, 'MA_700MA_std_mean']).mean()
    X.loc[seg_id,'MA_400MA_std_mean'] = xc.rolling(window=400).std().mean()
    X.loc[seg_id,'MA_400MA_BB_high_mean'] = (X.loc[seg_id, 'Moving_average_700_mean'] + no_of_std * X.loc[seg_id, 'MA_400MA_std_mean']).mean()
    X.loc[seg_id,'MA_400MA_BB_low_mean'] = (X.loc[seg_id, 'Moving_average_700_mean'] - no_of_std * X.loc[seg_id, 'MA_400MA_std_mean']).mean()
    X.loc[seg_id,'MA_1000MA_std_mean'] = xc.rolling(window=1000).std().mean()
    X.drop('Moving_average_700_mean', axis=1, inplace=True)
    
    #rolling features
    for w in [10, 50, 100, 1000]:
        x_roll_abs_mean = xc.abs().rolling(w).mean().dropna().values
        x_roll_mean = xc.rolling(w).mean().dropna().values
        x_roll_std = xc.rolling(w).std().dropna().values
        x_roll_min = xc.rolling(w).min().dropna().values
        x_roll_max = xc.rolling(w).max().dropna().values
        
        X.loc['ave_roll_std_' + str(w)] = x_roll_std.mean()
        X.loc['std_roll_std_' + str(w)] = x_roll_std.std()
        X.loc['max_roll_std_' + str(w)] = x_roll_std.max()
        X.loc['min_roll_std_' + str(w)] = x_roll_std.min()
        X.loc['q01_roll_std_' + str(w)] = np.quantile(x_roll_std, 0.01)
        X.loc['q05_roll_std_' + str(w)] = np.quantile(x_roll_std, 0.05)
        X.loc['q10_roll_std_' + str(w)] = np.quantile(x_roll_std, 0.10)
        X.loc['q95_roll_std_' + str(w)] = np.quantile(x_roll_std, 0.95)
        X.loc['q99_roll_std_' + str(w)] = np.quantile(x_roll_std, 0.99)
        
        X.loc['ave_roll_mean_' + str(w)] = x_roll_mean.mean()
        X.loc['std_roll_mean_' + str(w)] = x_roll_mean.std()
        X.loc['max_roll_mean_' + str(w)] = x_roll_mean.max()
        X.loc['min_roll_mean_' + str(w)] = x_roll_mean.min()
        X.loc['q05_roll_mean_' + str(w)] = np.quantile(x_roll_mean, 0.05)
        X.loc['q95_roll_mean_' + str(w)] = np.quantile(x_roll_mean, 0.95)
        
        X.loc['ave_roll_abs_mean_' + str(w)] = x_roll_abs_mean.mean()
        X.loc['std_roll_abs_mean_' + str(w)] = x_roll_abs_mean.std()
        X.loc['q05_roll_abs_mean_' + str(w)] = np.quantile(x_roll_abs_mean, 0.05)
        X.loc['q95_roll_abs_mean_' + str(w)] = np.quantile(x_roll_abs_mean, 0.95)
        
        X.loc['std_roll_min_' + str(w)] = x_roll_min.std()
        X.loc['max_roll_min_' + str(w)] = x_roll_min.max()
        X.loc['q05_roll_min_' + str(w)] = np.quantile(x_roll_min, 0.05)
        X.loc['q95_roll_min_' + str(w)] = np.quantile(x_roll_min, 0.95)

        X.loc['std_roll_max_' + str(w)] = x_roll_max.std()
        X.loc['min_roll_max_' + str(w)] = x_roll_max.min()
        X.loc['q05_roll_max_' + str(w)] = np.quantile(x_roll_max, 0.05)
        X.loc['q95_roll_max_' + str(w)] = np.quantile(x_roll_max, 0.95)

In [20]:
for seg_id in tqdm_notebook(range(segments)):
    seg = train_df.iloc[seg_id*rows:seg_id*rows+rows]
    create_features(seg_id, seg, train_X)
    train_y.loc[seg_id, 'time_to_failure'] = seg['time_to_failure'].values[-1]

In [21]:
train_y.head()

,time_to_failure
0,1.4307972193
1,1.3914989233
2,1.3531961441
3,1.3137978315
4,1.2743995190


In [14]:
train_X.head()

,mean,std,max,min,sum,mad,kurt,skew,med,abs_mean,q95,q99,q05,q01,Rmean,Rstd,Rmax,Rmin,Imean,Istd,Imax,Imin,std_first_50000,std_last_50000,std_first_25000,std_last_25000,std_first_10000,std_last_10000
0,4.8841133333,5.1011061306,104.0000000000,-98.0000000000,"732,617.0000000000",3.2634013568,33.6624812935,-0.0240611666,5.0000000000,5.5765666667,11.0000000000,18.0000000000,-2.0000000000,-8.0000000000,12.0000000000,"2,349.8114818002","732,617.0000000000","-20,121.1541712349",-0.0000000000,"1,399.8546353104","23,432.7194327180","-23,432.7194327180",6.4885518890,3.6646634202,7.9291837827,3.7913144645,11.2071511109,4.3614065193
1,4.7257666667,6.5888237819,181.0000000000,-154.0000000000,"708,865.0000000000",3.5743018511,98.7585171787,0.3905605044,5.0000000000,5.7341666667,12.0000000000,21.0000000000,-2.0000000000,-11.0000000000,5.0000000000,"2,566.0322484334","708,865.0000000000","-31,056.6750764376",0.0000000000,"1,810.3122658812","27,236.1805860738","-27,236.1805860737",7.3052326978,5.4930705003,8.7674684590,4.4858582141,3.9767501322,3.6678901267
2,4.9063933333,6.9673970335,140.0000000000,-106.0000000000,"735,959.0000000000",3.9484113621,33.5552114069,0.2173905614,5.0000000000,6.1526466667,13.0000000000,26.0000000000,-3.0000000000,-15.0000000000,5.0000000000,"2,683.5490493002","735,959.0000000000","-27,654.5570674995",0.0000000000,"1,921.2205755717","30,073.4970661712","-30,073.4970661712",6.1048364573,8.6036956138,6.9764512661,11.0572119819,8.4547171605,9.4939831663
3,4.9022400000,6.9223051872,197.0000000000,-199.0000000000,"735,336.0000000000",3.6471170688,116.5481716876,0.7572775364,5.0000000000,5.9339600000,12.0000000000,22.0000000000,-2.0000000000,-12.0000000000,5.0000000000,"2,685.7885248098","735,336.0000000000","-25,622.3936037852",-0.0000000000,"1,891.8263662398","27,380.3214706235","-27,380.3214706235",6.2381094757,5.6524418998,6.0865795452,6.1566031764,6.8661772100,4.3644299255
4,4.9087200000,7.3011101898,145.0000000000,-126.0000000000,"736,308.0000000000",3.8260516235,52.9779048344,0.0645310693,5.0000000000,6.1105866667,12.0000000000,26.0000000000,-2.0000000000,-15.0000000000,12.0000000000,"2,761.7157710380","736,308.0000000000","-26,271.0751169634",-0.0000000000,"1,995.7429694227","27,503.0452798519","-27,503.0452798519",5.3238295162,7.6945060676,6.3365758672,8.1365643937,5.1645936000,11.4048996016
